In [15]:
#cheat sheet here: https://crypto.stanford.edu/~dabo/cs255/handouts/numth1.pdf

import numpy as np
import pandas as pd

p=7

In [60]:
df = pd.DataFrame(data={'g': np.arange(p)})

# fermat's little theorem: for any element 0<g<p, g^(p-1) = 1 mod p
# proof here: https://en.wikipedia.org/wiki/Proofs_of_Fermat%27s_little_theorem
df['g^(p-1)'] = df['g']**(p-1) % p

# inverse is element x such that g*x = 1 mod p
# formula is x = g^(p-2)
# proof: g^(p-2) * g = g^(p-1) = 1 mod p from fermat's little theorem
# also proves that all elements besides 0 are invertible
df['g^-1'] = df['g']**(p-2) % p
df['(g^-1) * g'] = df['g^-1'] * df['g'] % p

# can solve equations of the form a*x = b mod p where a,b are known and greater than zero, and x is unknown
# solution: x = b*(a^-1) = b*a^(p-2) mod p
for b in range(1, p):
    for a in range(1, p):
        x = b*a**(p-2) % p
        #print(f'a: {a}, b: {b}, x: {x}, a*x: {a*x%p}')

# Zp* is the group of all invertible elements mod p, eg {1, ..., p-1}
# Zp* is a cyclic group: there exists some g such that Zp* = {1, g, g^2, ..., g^(p-2)} (note that g^(p-1) goes back to 1)
Zp = pd.DataFrame()
for g in range(1, p):
    Zp[f'g={g}'] = [g**i % p for i in range(0, p-1)]

# not every g is a valid generator, for example 2 is not valid {1,4,2} but 3 is valid {1,3,2,6,4,5}
# considered valid if it hits every point in Zp*
df['|Zg|'] = [np.nan] + list(Zp.nunique().values)
df['valid generator'] = df['|Zg|'] == p-1

# order of g is smallest integer a such that g^a = 1 mod p
# order is equal to the already-calculated |Zg|
# denoted ord_p(g), eg ord_7(2)=3, ord_7(3) = 6

# Lagrange's Theorem: For all g in Zp, ord_p(g) divides p-1
# since p-1 = 6, all of the orders should be divisors of 6, which we can clearly see (1, 3, 6, 3, 6, 2)

# If the factorization of p-1 is known, then there is a simple and efficient algorithm to determine ord_p(g) for all g

In [59]:
print(df)
print(Zp)

   g  g^(p-1)  g^-1  (g^-1) * g  |Zg|  valid generator
0  0        0     0           0   NaN            False
1  1        1     1           1   1.0            False
2  2        1     4           1   3.0            False
3  3        1     5           1   6.0             True
4  4        1     2           1   3.0            False
5  5        1     3           1   6.0             True
6  6        1     6           1   2.0            False
   g=1  g=2  g=3  g=4  g=5  g=6
0    1    1    1    1    1    1
1    1    2    3    4    5    6
2    1    4    2    2    4    1
3    1    1    6    1    6    6
4    1    2    4    4    2    1
5    1    4    5    2    3    6


In [65]:
# A square root of x is a number y such that x^2 = y mod p
df = pd.DataFrame(data={'x': np.arange(1,p)})
df['x^2'] = df['x']**2 % p
# can see visually that sqrt(1)=1 and 6, sqrt(2)=3 and 4, sqrt(4)=2 and 5

# an element is called a Quadratic Residue (QR) if it has a square root
# if an element has at least one square root, then it must have exactly 2 square roots
# proof: if x^2 = y^2 mod p, then 0 = x^2 - y^2 mod p = (x-y)(x+y) mod p => either x=y or x=-y mod p

# Euler's Theorem: an element x is a QR if and only if x^((p-1)/2) = 1 mod p
# I guess this makes sense, since x^2 = x^(p-1) = 1 mod p from Fermat's Little Theorem
df['x^(p-1)/2'] = df['x']**((p-1)/2) % p
# the numbers where this is 1 are (1,2,4) as expected from visual inspection

# g^(p-1)/2 must be either 1 or -1. Proof:
# g^(p-1) is 1 mod p from Fermat's Little Theorem
# (g^(p-1)/2)^2 = g^(p-1) = 1 mod p
# therefore g^(p-1)/2 is the square root of 1 mod p, which is either 1 or -1 mod p

# Legendre Symbol: (x | p) = {1 if x is a QR of p, -1 if x is not a QR of p, 0 if x=0}
# Legendre Symbol calculated using (x | p) = x^(p-1)/2 mod p


In [66]:
df

,x,x^2,x^(p-1)/2
0,1,1,1.0
1,2,4,1.0
2,3,2,6.0
3,4,2,1.0
4,5,4,6.0
5,6,1,6.0
